In [864]:
import pandas as pd
import pyreadstat
import os
import numpy as np
from IPython.display import HTML, display
PATH = "/Users/renjiege/Documents/taskdata/Worldbank/"
dta = "PTA2_test.dta"
excel = "STRI_ScoreCard_1701.xlsx"

### Import Data

In [865]:
table_names = ['ScoreCard_2016', 'ScoreAggregateByFunc_2016', 'ScoreAggregate_2016', 
               'ScoreNumeric_2016', 'ParentChildCode_2016','ParentChildCodeNumeric_2016']
pta = pd.read_stata(f'{PATH}{dta}')
pta['Measure_ID'] = pta.Measure_ID.astype(np.int64)

In [866]:
pta.Answer.value_counts()

yes    3028
no      263
70       12
5        10
49        3
Name: Answer, dtype: int64

In [867]:
tables = [pd.read_excel(f'{PATH}{excel}', sheet_name=f'{fname}') for fname in table_names]
score_card, aggregate_by_func, aggregate, numeric, parent_child, parent_child_numeric = tables

In [868]:
pd.options.display.max_colwidth = 80
pd.options.display.max_columns = 30
pd.options.display.max_rows = 100

In [869]:
score_card = score_card[["Code", "Answer", "Category", "Subcategory", "Subsector", "Score"]]
score_card[score_card.Code==149][["Code", "Answer", "Category", "Subcategory", "Subsector", "Score"]]

,Code,Answer,Category,Subcategory,Subsector,Score
26,149,yes,A -Market entry,A2,;10874;10875;10891;10892;10873;10880;10882;10881;10883;152;174;169;,0.125
27,149,yes,A -Market entry,A2,;31;30;10904;207;206;136;10903;10901;10902;10897;10898;,0.250


### Merging and Data cleaning

In [870]:
aggregate = aggregate.rename(columns={'Subsector ': 'Subsector'})

In [871]:
joined = pta.merge(score_card, left_on="Measure_ID", right_on="Code", how='outer', suffixes=('', '_s'))

In [872]:
joined['Sectorfine_ID'] = pta.Sectorfine_ID.astype(int).astype(str)

**Note**: Some measures have different scores for different sectors (Measure 25 for example). When we use the outer merge, these measures are mapped to wrong sectors. Thus we need to drop these measures where subsector information do not match. 

In [873]:
dup_code = []
for t in joined.Code.unique():
    if len(joined[joined.Code==t]["Score"].unique())>1:
        dup_code.append(str(t))
dup_code

['25', '149', '136', '39', '144']

In [874]:
joined['tag1'] = joined.apply(lambda x: str(x.Sectorfine_ID) in str(x.Subsector), axis=1)

In [875]:
joined['tag2'] = joined.apply(lambda x: str(x.Code) in dup_code, axis=1)

In [876]:
score_card[score_card.Code==25]

,Code,Answer,Category,Subcategory,Subsector,Score
81,25,no,A -Market entry,A1,;31;30;10904;207;206;136;10901;10902;10903;10880;10882;10881;10883;10897;108...,0.50
82,25,no,A -Market entry,A1,;10874;10875;10891;10892;10873;,0.25


In [877]:
joined = joined[joined.Sectorfine_ID.isnull()==False]
joined[joined.Code==25][["Code", "Answer", "Category", "Sectorfine", "Sectorfine_ID", "M1", "M2", "M3","M4", "Subcategory", "Subsector", "Score", "tag1", "tag2"]]

,Code,Answer,Category,Sectorfine,Sectorfine_ID,M1,M2,M3,M4,Subcategory,Subsector,Score,tag1,tag2
2591,25,no,A -Market entry,1.A.a.(b),10873,0.0,0.0,1.0,0.0,A1,;31;30;10904;207;206;136;10901;10902;10903;10880;10882;10881;10883;10897;108...,0.50,False,True
2592,25,no,A -Market entry,1.A.a.(b),10891,0.0,0.0,1.0,0.0,A1,;10874;10875;10891;10892;10873;,0.25,True,True
2593,25,no,A -Market entry,1.A.a.(a.1),10892,0.0,0.0,1.0,0.0,A1,;31;30;10904;207;206;136;10901;10902;10903;10880;10882;10881;10883;10897;108...,0.50,False,True
2594,25,no,A -Market entry,1.A.a.(a.1),10874,0.0,0.0,1.0,0.0,A1,;10874;10875;10891;10892;10873;,0.25,True,True
2595,25,no,A -Market entry,1.A.a.(a.2),10874,0.0,0.0,1.0,0.0,A1,;31;30;10904;207;206;136;10901;10902;10903;10880;10882;10881;10883;10897;108...,0.50,False,True
2596,25,no,A -Market entry,1.A.a.(a.2),10875,0.0,0.0,1.0,0.0,A1,;10874;10875;10891;10892;10873;,0.25,True,True
2597,25,no,A -Market entry,1.A.a.(a.1),10873,0.0,0.0,1.0,0.0,A1,;31;30;10904;207;206;136;10901;10902;10903;10880;10882;10881;10883;10897;108...,0.50,False,True
2598,25,no,A -Market entry,1.A.a.(a.1),10873,0.0,0.0,1.0,0.0,A1,;10874;10875;10891;10892;10873;,0.25,True,True
2599,25,no,A -Market entry,1.A.a.(a.2),10874,0.0,0.0,1.0,0.0,A1,;31;30;10904;207;206;136;10901;10902;10903;10880;10882;10881;10883;10897;108...,0.50,False,True
2600,25,no,A -Market entry,1.A.a.(a.2),10874,0.0,0.0,1.0,0.0,A1,;10874;10875;10891;10892;10873;,0.25,True,True


In [878]:
joined = joined.drop(joined[(joined.tag1==False) & (joined.tag2==True)].index)

In [879]:
joined = joined.drop(['tag1', 'tag2', 'Subsector'], axis=1)

In [880]:
joined.Answer_s.value_counts()

yes        2804
no          186
numeric      14
Name: Answer_s, dtype: int64

In [881]:
# parent_child[parent_child.columns.drop(list(parent_child.filter(regex='Unnamed')))]

The scorecard has some three main functions that modify the scores under certain conditions.

### Parent-child function

First define a function that finds parent code index given the child index, then use this function to locate the parent answer and compare it with the answer in the parent_child spreadsheet. Generate a dummy to indicate whether the child measure is active.

In [882]:
def parent(key):
    Dict = {x: y for x,y in zip(parent_child.ChildCode, parent_child.ParentCode)}
    return Dict[key]

def default(key):
    Dict = {x: y for x,y in zip(parent_child.ChildCode, parent_child.ParentAnswer)}
    return Dict[key]

In [883]:
def find_parent_index(child_index):
    child_code = joined.loc[(child_index), ['Code']].values[0]
    parent_code = parent(child_code)
    sector_id = str(joined.loc[(child_index), ['Sectorfine_ID']].values[0])
    regime = joined.loc[(child_index), ['regime_id']].values[0]
    parent_index = joined[(joined.Sectorfine_ID==sector_id) & (joined.regime_id==regime) & (joined.Code==parent_code)].index
#     print(parent_code)
#     print(sector_id)
#     print(regime)
    return parent_index

In [884]:
joined['active']=1
for index, row in joined.iterrows():
    each = row['Code']
    child_index = index
    if each in list(parent_child.ChildCode):
        default_answer = default(each)    
#         print(each)
        parent_index = find_parent_index(child_index)
#         print(parent_index)
        if parent_index.values.any()==True:  # i.e., if parent code exists
            parent_answer = joined.loc[(parent_index), ['Answer']].values[0]
            if parent_answer!=default_answer:
                joined.loc[joined.Code==each,'active']=0

### Numeric function

First define a function that returns score given the numeric answer and associated measure code, subsector, and mode. Then apply this function to each row in the joined dataframe. 

In [885]:
def get_numeric_score(code, answer, subsector):
    for idx, row in numeric.iterrows():
        Max = int(row['Max'])
        Min = int(row['Min'])
        Code = row['Code']
        Subsector = row['Subsector_numeric']
#         Mode = row['Mode']
        if answer<=Max and answer>=Min and code==Code and (str(subsector) in Subsector):
                score = numeric.loc[(idx), 'Score']
                return score

In [886]:
get_numeric_score(27, 26, 206)

0.25

In [887]:
joined['Score_s'] = joined.Score
joined.loc[joined.Answer_s!=joined.Answer, "Score_s"] =0

In [888]:
for idx, row in joined.iterrows():
    if row['Answer_s'] == 'numeric':
#         print(idx)
        answer = int(row['Answer'])
        code = row['Code']
        subsector= row['Sectorfine_ID']
        Score = get_numeric_score(code, answer, subsector)
        joined.loc[(idx), 'Score_s'] = Score

In [889]:
# joined.loc[((joined.Code==137) | (joined.Code==139))]

### Aggregation function

There are three main steps involved aggregating subcode into a pseudo code. First, define a function "get_pseudo_score", which outputs the pseudo code and its corresponding score given data. Then define a function, "get_pseudo_data", which given dataframe, produces the data that need to be added to the dataframe. Lastly, we update the df by group at regime-country-sector level. 


The following test sample is a good subsample to examine the performance of the code. It contains two sets of codes that need to be aggregated into one pseudo code. 

In [890]:
test = joined.loc[(joined.Regime=="ASEAN - Korea, Republic of") & (joined.Country=="Brunei Darussalam") & (joined.Sectorfine_ID=="10873")] \
    [['Regime', 'Country', 'Sectorfine_ID', 'Subcategory', 'Answer', 'Code', 'Score', 'Score_s', 'M1', 'M2', 'M3', 'M4']]
df = joined[['Regime', 'Country', 'Sectorfine_ID', 'Subcategory', 'Answer', 'Code', 'Score', 'Score_s', 'M1', 'M2', 'M3', 'M4']].reset_index(drop=True)
df = df.drop(df[df.Country==""].index)

In [891]:
def get_pseudo_score(data):
    dic, subcategory, subsector, mode = data
    for idx, row in aggregate.iterrows():
        sub = row['Subsector']
        code = row['Code']
        answer = row['Answer']
        Pseudo_code = None
        score = None
        if (subcategory==row['Subcategory']) and (subsector in sub) and (row['Mode'] in mode):
            if answer=='no':
                list_code = [y for x,y in zip(dic.values(), dic.keys()) if x =='no']
                concat_code = ','.join(sorted(list_code, key=float))
                if concat_code == code:
                    Pseudo_code = aggregate.loc[(idx), 'PseudoCode']
                    score = aggregate.loc[(idx), 'Score']
                    return Pseudo_code, score
            else:
                list_code = [y for x,y in zip(dic.values(), dic.keys()) if x =='yes']
                concat_code = ','.join(sorted(list_code, key=float))
                if concat_code == code:
                    Pseudo_code = aggregate.loc[(idx), 'PseudoCode']
                    score = aggregate.loc[(idx), 'Score']
                    return Pseudo_code, score

In [892]:
dic = {'137': 'no', '139': 'no'}
get_pseudo_score([dic, 'A1', '10904', ['M1','','M3','M4']])

('9991C', 0.5)

In [893]:
list_single = [x for x in list(aggregate.Code.unique()) if (',' not in x)]
list_multiple = [list_single[n:n+3] for n in range(0, len(list_single), 3)]

In [894]:
def get_related_codes(x):
    for y in list_multiple:
        if str(x) in y:
            other_code = [s for s in y if s != str(x)]
            return other_code

In [895]:
get_related_codes(137)

['136', '139']

In [896]:
def get_pseudo_data(df): 
    
    code_to_answer = {str(s): str(t) for s,t in zip(df.Code, df.Answer)}
    unique_code = list(df.Code.unique().astype(str))
    
    pseudo_data_list = []
    for idx, row in df.iterrows():
        
        subsector = row['Sectorfine_ID']
        subcategory = row['Subcategory']
        M1 = int(row['M1'])*'M1'
        M2 = int(row['M2'])*'M2'
        M3 = int(row['M3'])*'M3'
        M4 = int(row['M4'])*'M4'
        mode = [M1, M2, M3, M4]
    
        if str(row['Code']) in list_single:
            answer_1 = row['Answer']
            related_codes = get_related_codes(row['Code'])
            all_codes = [str(row['Code'])] + related_codes
            answer={str(row['Code']): answer_1}
            
            for i, j in enumerate(related_codes):
                if j in unique_code: 
                    other_answer = {j: code_to_answer[j]}
                    answer.update(other_answer)
            
            data = [answer, subcategory, subsector, mode] 
            pseudo_code, score = get_pseudo_score(data)
            pseudo_data = (pseudo_code, score, subcategory, ','.join(list(answer.values())), subsector, M1, M2, M3, M4)
            pseudo_data_list.append(pseudo_data) 
     
    return list(set(pseudo_data_list))

In [897]:
get_pseudo_data(test)

[('9991Ab', 0.125, 'A2', 'yes', '10873', '', '', '', 'M4')]

In [898]:
def add_pseudo_data(df):
    pseudo_data = get_pseudo_data(df)
    if pseudo_data:
        for each in pseudo_data:
            last_row = df.iloc[[-1]]
            data_list = list(each)
            last_row['Subcategory'] = data_list[2]
            last_row['Score_s'] = data_list[1]
            last_row['Code'] = data_list[0]
            last_row['M1'] = 1 if data_list[-4] else 0
            last_row['M2'] = 1 if data_list[-3] else 0
            last_row['M3'] = 1 if data_list[-2] else 0
            last_row['M4'] = 1 if data_list[-1] else 0
            df = df.append(last_row)
        return df
    return df

In [899]:
# add_pseudo_data(test)

In [900]:
df = df.groupby(['Regime', 'Country','Sectorfine_ID']).apply(add_pseudo_data).reset_index(drop=True)

In [901]:
for each in list_single:
    df = df.drop(df[df.Code==int(each)].index)

In [902]:
df.Code.unique()

array([12, 19, 15, 82, 227, 24, 38, 827, 25, 40, 54, 62, '9991Ab',
       '9991A', 66, 300, 13, 89, 75, 860], dtype=object)

In [903]:
df = df.drop(['Score'], axis=1)

### Double Check Data Issues

In [904]:
df

,Regime,Country,Sectorfine_ID,Subcategory,Answer,Code,Score_s,M1,M2,M3,M4
0,,Russian Federation,10873,A1,yes,12,1.00,1.0,0.0,0.0,0.0
1,,Russian Federation,10873,A1,yes,12,1.00,1.0,0.0,0.0,0.0
2,,Russian Federation,10873,A1,yes,12,1.00,1.0,0.0,0.0,0.0
3,,Russian Federation,10873,A1,yes,12,1.00,1.0,0.0,0.0,0.0
4,,Russian Federation,10873,A1,yes,12,1.00,1.0,0.0,0.0,0.0
5,,Russian Federation,10874,A1,yes,19,1.00,0.0,0.0,1.0,0.0
6,,Russian Federation,10875,A1,yes,19,1.00,0.0,0.0,1.0,0.0
7,,Russian Federation,10891,A1,yes,19,1.00,0.0,0.0,1.0,0.0
8,,Russian Federation,10892,A1,yes,19,1.00,0.0,0.0,1.0,0.0
9,,Russian Federation,10892,A1,yes,19,1.00,0.0,0.0,1.0,0.0


In [905]:
df[df.Code==24]

,Regime,Country,Sectorfine_ID,Subcategory,Answer,Code,Score_s,M1,M2,M3,M4
124,ASEAN - Australia - New Zealand,Indonesia,10873,A1,49,24,0.1250,0.0,0.0,1.0,0.0
131,ASEAN - Australia - New Zealand,Indonesia,10874,A1,49,24,0.1250,0.0,0.0,1.0,0.0
138,ASEAN - Australia - New Zealand,Indonesia,10875,A1,49,24,0.1250,0.0,0.0,1.0,0.0
957,Costa Rica - Peru,Costa Rica,10873,A1,5,24,0.1875,1.0,0.0,1.0,0.0
961,Costa Rica - Peru,Costa Rica,10874,A1,5,24,0.1875,1.0,0.0,1.0,0.0
962,Costa Rica - Peru,Costa Rica,10874,A1,5,24,0.1875,1.0,0.0,1.0,0.0
963,Costa Rica - Peru,Costa Rica,10874,A1,5,24,0.1875,1.0,0.0,1.0,0.0
964,Costa Rica - Peru,Costa Rica,10875,A1,5,24,0.1875,1.0,0.0,1.0,0.0
1796,Gulf Cooperation Council (GCC) - Singapore,Oman,10874,A1,70,24,0.0625,0.0,0.0,1.0,0.0
1797,Gulf Cooperation Council (GCC) - Singapore,Oman,10875,A1,70,24,0.0625,0.0,0.0,1.0,0.0


In [906]:
os.chdir(PATH)
df.to_csv('PTA2_test_02192019.csv', sep=',')

In [907]:
df = df.drop_duplicates()

In [908]:
df

,Regime,Country,Sectorfine_ID,Subcategory,Answer,Code,Score_s,M1,M2,M3,M4
0,,Russian Federation,10873,A1,yes,12,1.0000,1.0,0.0,0.0,0.0
5,,Russian Federation,10874,A1,yes,19,1.0000,0.0,0.0,1.0,0.0
6,,Russian Federation,10875,A1,yes,19,1.0000,0.0,0.0,1.0,0.0
7,,Russian Federation,10891,A1,yes,19,1.0000,0.0,0.0,1.0,0.0
8,,Russian Federation,10892,A1,yes,19,1.0000,0.0,0.0,1.0,0.0
10,ASEAN,Brunei Darussalam,10873,A1,yes,19,1.0000,0.0,0.0,1.0,0.0
12,ASEAN,Brunei Darussalam,10873,A1,yes,12,1.0000,1.0,0.0,0.0,0.0
17,ASEAN,Brunei Darussalam,10874,A1,yes,19,1.0000,0.0,0.0,1.0,0.0
18,ASEAN,Brunei Darussalam,10875,A1,yes,19,1.0000,0.0,0.0,1.0,0.0
20,ASEAN,Cambodia,10873,A1,yes,19,1.0000,0.0,0.0,1.0,0.0


In [910]:
df.to_csv('PTA2_test_rd_02192019.csv', sep=',')